In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
WAIT = 3
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

In [2]:
# css 찾을때 까지 10초대기
def time_wait(num, code,driver):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [3]:
def find_search_input(key_word,driver):
    # 검색창 찾기
    driver.switch_to.default_content()
    time_wait(2, 'div.input_box > input.input_search',driver)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    driver.implicitly_wait(3)
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    sleep(5)
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [4]:
# frame 변경 메소드
def switch_frame(frame,driver):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [5]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [6]:
# 페이지 다운
def page_down(num,driver):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [7]:
def find_review_btn(driver):
    sleep(2)
    try:
        menu_list = driver.find_elements(By.CSS_SELECTOR,'a.tpj9w')
        if len(menu_list) ==7:
            return menu_list[-1]
        else:
            return menu_list[-2]
    except:
        print('리뷰가 존재하지 않습니다.')

# 메뉴&리뷰 크롤링 함수

In [8]:
def crawling_menu_review(driver):
    #-----리뷰 버튼 클릭-------
    find_review_btn(driver).click()
    SCROLL_PAUSE_SEC = 1
    sleep(1)
    page_down(1,driver)
    driver.implicitly_wait(WAIT)
    #----------------메뉴, 크롤링 블록----------------
    menu_list = list()
    review_list = list()
    try:
        # --------------메뉴 크롤링 블록--------------------------
        try :
            #메뉴 요소 찾기
            menu_elements  = driver.find_elements(By.CSS_SELECTOR,'div.place_fixed_subtab > div > div:nth-child(1) > div > div > div > span > a > span:first-child')
            sleep(SCROLL_PAUSE_SEC)
            driver.implicitly_wait(WAIT)
            try:
                #왼쪽 버튼 찾기
                left_btn = driver.find_element(By.CSS_SELECTOR,'a.PznE8.aNw43')
                driver.implicitly_wait(WAIT)
                left_btn.click()
                driver.implicitly_wait(WAIT)
            except:
                pass

            try:
                #오른족 버튼 찾기
                right_btn = driver.find_element(By.CSS_SELECTOR, 'div.place_fixed_subtab > div > div:nth-child(1) > div > div > a.ZCqf_.aNw43')
                driver.implicitly_wait(WAIT)
            except:
                pass
            #-----------여기까지-----------
            i = 0
            # menu_elentents가 존재하지 않는다면 except로 갈 것이다.
            try: 
                while True:
                    if menu_elements[i].text != '': #요소가 존재하고 메뉴 텍스트도 보일때
                        menu_list.append(menu_elements[i].text)
                        if i == len(menu_elements)-1: # 메뉴 리스트 길이면 나온나
                            break
                        i+=1
                    else:
                        right_btn.click()
                        sleep(SCROLL_PAUSE_SEC)
                        driver.implicitly_wait(WAIT)
            except:
                pass

            #last_scroll = driver.execute_script("return document.body.scrollHeight")
            #view_more_count = 0

        except:
            pass

        # --------------리뷰 크롤링 블록--------------------------
        try:
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(1)
                driver.implicitly_wait(SCROLL_PAUSE_SEC)

                #if view_more_count ==19:
                    #break
                try :
                    #더보기 버튼 클릭
                    driver.implicitly_wait(WAIT)
                    driver.find_element(By.CSS_SELECTOR,'a.fvwqf').click()
                    #view_more_count += 1

                except:
                    #더보기 버튼이 없다면 while문을 빠져나온다.
                    break
                    # 끝까지 스크롤 다운

                #new_scroll = driver.execute_script("return document.body.scrollHeight")
                #driver.implicitly_wait(WAIT)
                #if last_scroll == new_scroll:
                #    break
                #else:
                #    last_scroll= new_scroll

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            #가게 리스트 마다 방문자 리뷰 가져오기
            review_elements = driver.find_elements(By.CSS_SELECTOR,"span.zPfVt")
            for i in review_elements:
                review_list.append(i.text)
        except:
            pass
    
    except:
        pass
    
    return menu_list, review_list

# 전체 크롤링 함수

In [52]:
def crawling(driver, i, store_elements,review_dic):
    driver.implicitly_wait(1)
    store_elements[i].click() #li의 스토어를 클릭
    driver.implicitly_wait(WAIT) # 2초 휴식
    try:
        #다른 iframe으로 변경
        switch_frame('entryIframe',driver)
        time_wait(2, '.Fc1rA',driver)

        # -----매장명 가져오기-----
        store_name = driver.find_element(By.CSS_SELECTOR,'.Fc1rA').text
        driver.implicitly_wait(2)
        review_dic[store_name] = dict()
        
        # -----평점 가져오기-----
        try :
            rank = driver.find_element(By.CSS_SELECTOR,'div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em')
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['rank'] = rank.text
        except:
            pass

        # -----위치 가져오기-----
        try : 
            location = driver.find_element(By.CSS_SELECTOR, 'span.IH7VW').text
            driver.implicitly_wait(WAIT)
            review_dic[store_name]['location'] = location
        except:
            pass
        
        
        #---------------문제--------------
        memu_list, review_list = crawling_menu_review(driver)
        
        # 메뉴 크롤링
        review_dic[store_name]['menu'] = menu_list
        
        # 방문자 리뷰 크롤링
        review_dic[store_name]['review'] = review_list
        driver.implicitly_wait(WAIT)
        
    except:
        pass
    
    memu_list, review_list = crawling_menu_review(driver)
        
    # 메뉴 크롤링
    review_dic[store_name]['menu'] = memu_list

    # 방문자 리뷰 크롤링
    review_dic[store_name]['review'] = review_list
    driver.implicitly_wait(WAIT)
    
    return store_name

# 메인 함수

In [53]:
def main(key_word, page_num):
    review_dic = dict()
    page_num = page_num-1
    refresh_count = 0
    
    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome('./chromedriver')
    driver.get(url)
    driver.implicitly_wait(2)

    # 검색어 입력
    find_search_input(key_word,driver=driver)
    driver.implicitly_wait(2)

    #프레임 변경
    switch_frame('searchIframe',driver)
    driver.implicitly_wait(WAIT)

    # 페이지 버튼 찾기
    pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
    if page_num <= 5:
        pages[page_num].click()
        driver.implicitly_wait(WAIT)
    else:
        driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)
        driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
        driver.implicitly_wait(WAIT)

    driver.implicitly_wait(WAIT)
    page_down(40,driver)
    sleep(1)
    driver.implicitly_wait(1)
    page_down(5,driver)
    sleep(2)
    driver.execute_script("window.scrollTo(0, 0);")
    sleep(2)
    store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
    print('total store_list : ', len(store_elements)) #매장리스트 접근
    # 체크포인트
    ck_pt_idx = 0
    
    while True:
        while True:
            try: #이상한 오류가 뜰경우 현재위치 까지 저장 하고, 크롤링 인덱스 반환
                store_name = crawling(driver, ck_pt_idx, store_elements, review_dic)
                print('현재 크롤링 중인 매장 인덱스와 매장명 : ', ck_pt_idx, store_name)
                ck_pt_idx +=1
                switch_frame('searchIframe',driver)

                #한 refresh를 벗어나는 조건문
                if len(review_dic[store_name]['review']) >= 500:
                    break
            except:
                # json 파일로 저장
                with open(f'./naver/naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
                    json.dump(review_dic, f, indent=4, ensure_ascii=False) 
                return ck_pt_idx
                    
        
        # 페이지의 크롤링을 나오는 조건문
        if ck_pt_idx == len(store_elements)-1:
            break
            
        driver.refresh()
        driver.implicitly_wait(1)
        switch_frame('searchIframe',driver)
        sleep(0.5)
        page_down(40,driver)
        sleep(1)
        driver.implicitly_wait(1)
        page_down(5,driver)
        sleep(2)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
        
        
        

    # json 파일로 저장
    with open(f'./Crawling2/naver_{key_word}_review_dic_page{page_num+1}.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)        
    

In [54]:
main('울산 횟집', 1)

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_8592\847684578.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  44
현재 크롤링 중인 매장 인덱스와 매장명 :  0 떡바우횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  1 웰컴도매횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  2 일차로물회
현재 크롤링 중인 매장 인덱스와 매장명 :  3 떡바우횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  4 동남횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  5 경북횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  6 김대통의대통물회&횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  7 삼천포횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  8 육일횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  9 뽀빠이물회
현재 크롤링 중인 매장 인덱스와 매장명 :  10 대복횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  11 강동회수산
현재 크롤링 중인 매장 인덱스와 매장명 :  12 독도참가자미
현재 크롤링 중인 매장 인덱스와 매장명 :  13 삼태육경 회 타운
현재 크롤링 중인 매장 인덱스와 매장명 :  14 만원의행복횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  15 송가정횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  16 뱃고동횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  17 금바다횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  18 진주횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  19 금광횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  20 신명횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  21 떡바우횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  22 야무야무
현재 크롤링 중인 매장 인덱스와 매장명 :  23 방어진정자횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  24 해강횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  25 대게신도시횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  26 장원횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  27 진하자연산횟집
현재 크롤링 중인 매장 인덱스와 매장명 :  

42

In [55]:
with open('./naver/naver_울산 횟집_review_dic_page1.json', encoding='utf-8') as f:
    ex = json.load(f)
len(ex)

39

In [56]:
ex.keys()

dict_keys(['떡바우횟집', '웰컴도매횟집', '일차로물회', '동남횟집', '경북횟집', '김대통의대통물회&횟집', '삼천포횟집', '육일횟집', '뽀빠이물회', '대복횟집', '강동회수산', '독도참가자미', '삼태육경 회 타운', '만원의행복횟집', '송가정횟집', '뱃고동횟집', '금바다횟집', '진주횟집', '금광횟집', '신명횟집', '야무야무', '방어진정자횟집', '해강횟집', '대게신도시횟집', '장원횟집', '진하자연산횟집', '남일횟집', '갯바위횟집', '바닷가횟집', '미미정횟집고래고기', '청파', '일월횟집', '대어횟집', '후포미송', '섬뜰', '부산횟집', '수암사랑도횟집', '한양횟집', '동남물회'])

In [38]:
review_dic = dict()
key_word = '울산 횟집'
page_num  = 0
refresh_count = 0

url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)
driver.implicitly_wait(2)

# 검색어 입력
find_search_input(key_word,driver=driver)
driver.implicitly_wait(2)

#프레임 변경
switch_frame('searchIframe',driver)
driver.implicitly_wait(WAIT)

# 페이지 버튼 찾기
pages = driver.find_elements(By.CSS_SELECTOR,'a.mBN2s')
if page_num <= 5:
    pages[page_num].click()
    driver.implicitly_wait(WAIT)
else:
    driver.find_elements(By.CSS_SELECTOR,'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)
    driver.find_element(By.CSS_SELECTOR, 'div.zRM9F > a:nth-child(6)').click()
    driver.implicitly_wait(WAIT)

driver.implicitly_wait(WAIT)
page_down(40,driver)
sleep(1)
driver.implicitly_wait(1)
page_down(5,driver)
sleep(2)
driver.execute_script("window.scrollTo(0, 0);")
driver.implicitly_wait(WAIT)
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')
print('total store_list : ', len(store_elements)) #매장리스트 접근
# 체크포인트

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_8592\2411810518.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


total store_list :  44


In [39]:
switch_frame('searchIframe',driver)

In [40]:
store_name = crawling(driver, 43, store_elements, review_dic)

In [42]:
store_name

'동화회시장'

In [44]:
driver.refresh()

In [45]:
switch_frame('searchIframe',driver)

In [47]:
page_down(40,driver)

In [26]:
store_elements = driver.find_elements(By.CSS_SELECTOR,'div.ouxiq.icT4K > a:nth-child(1)')

In [28]:
store_name = crawling(driver, 40, store_elements, review_dic)

In [29]:
store_name

'수암사랑도횟집'

In [ ]:
crawling(driver, 4, store_elements, review_dic)

In [30]:
review_dic

{'떡바우횟집': {'rank': '4.41',
  'location': '울산 울주군 서생면 신암해안길 13 2층',
  'menu': ['물회', '매운탕', '밥집', '회덮밥', '농어', '광어'],
  'review': ['음식이 간결하고 맛있네요 ^^',
   '울산여행왔다가 근처 밥집찾아서 갔는데 넘 맛있게\n잘먹었어요~ 물회👍🏻👍🏻',
   '오랫만에 방문했는데 맛있게 잘 먹었습니다~',
   '물회 최고',
   '매운탕은 비리네요\n회덮밥은 먹을만해요',
   '친절하고 좋아요 ^^',
   '굿',
   '맛있어요',
   '굿',
   '맛있네요',
   '맛있네요',
   '신선합니다',
   '좋아요~',
   '좋음',
   '회가 신선합니다.',
   '가끔 가는곳',
   '맛있어요',
   '회 ㅡㅡ굿입니다 일하시는분들 모두친절하시구요',
   '굿',
   '가성비좋음',
   '횟밥 가성비 최고 추천',
   '알고 있는 맛',
   '굿',
   '농어랑 광어 포장해와서 맛있게 잘 먹었네요.\n농어도 광어도 맛 좋드라구요!!',
   '굿',
   '맛있어요',
   '맛있어요!',
   '😀😀😀😀😀',
   '좋아요',
   '맛있어요\n횟밥시키면 매운탕 도 같이 줘요!!!',
   '',
   'ㆍ',
   '회 굳']},
 '동화회시장': {'rank': '4.44',
  'location': '울산 남구 중앙로44번길 11',
  'menu': [],
  'review': ['오랜만에 방문 \n맛있게 잘 먹었어요',
   '주차장이없어서 아쉽지만 맛잇게 잘먹음',
   '좋아요',
   '동네 횟집\n룸이 있어 모임하기 좋아요.',
   '가족모임있었는데 음식 맛있고 아이 있으니 계란찜등 아이가 먹을 만한 음식도 빨리 챙겨주셔서 좋았어요',
   'ㅇ',
   '좋아요',
   '첫방문인데 아주 만족스럽네요. 재방문의사 있습니다. 번창하십시요',
   '최고체고',
   '좋아요',
   '맛나게 먹고 왔어요',


In [32]:
del review_dic